In [1]:
import re
import spacy
from spacy_langdetect import LanguageDetector
import en_core_web_sm
nlp = en_core_web_sm.load()
nlp.add_pipe(LanguageDetector(), name='language_detector', last=True)
import pandas as pd
import bson
import pymongo
from tqdm import tqdm
username = "tagger-admin"
password = "tvaiadmin"
db_client = pymongo.MongoClient('104.198.62.226', username=username,
                                password=password,
                                authSource='tags',
                                authMechanism='SCRAM-SHA-256', port=27017).tags

/home/ubuntu/anaconda3/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: thinc.extra.search.Beam size changed, may indicate binary incompatibility. Expected 112 from C header, got 120 from PyObject
  return f(*args, **kwds)


In [2]:
from fastai import *
from fastai.vision import *
from tagger import config, train_model, stringConverter
from tagger import predict

learner = load_learner("models", "guten_trash_P3_v1-server.pkl")

In [3]:
import aerospike
config_aerospike = {'hosts': [('35.228.136.58', 3000)]}
client = aerospike.client(config_aerospike).connect()
bert_data_key = ("ids_to_validate", "data", "bert_tags")

In [4]:
args_trash = {
    "labels": client.get(bert_data_key)[-1]["gutenTrash"],
    "model": learner,
}

In [5]:
def mongoRemover(bson_id, reason):        
    db_client["tweets_pipeline_v2"].update_one({"_id":bson.ObjectId(bson_id)},
                                        {"$set":{
                                                 "status":"deleted",
                                                 "GutenTagTrash": reason,
                                                }})

In [6]:
db = db_client["tweets_pipeline_v2"].find({"$or":[{"status": "pipelined"}, 
#                                                       {"status": "reserved"},                                                       
#                                                       {"status": "deleted"}, 
#                                                       {"status": "deleted_from_analytics"}
                                                 ],
                                                   "extracted_text": {"$exists": True},                                               
#                                                       "GutenTagTrash": {"$exists": False},

#                                                'tweet_timestamp': {'$gte': datetime(2020,2,4)} #algoStarts                                              
                                          }, no_cursor_timeout=True)
for i in tqdm(db, total = db.count()):
    text = i["extracted_text"]
    text = stringConverter(text, use_mister_proper=False)        
    result = predict(text, args_trash)
    _id = str(i["_id"])
    if result:
        print(_id, result)
        mongoRemover(_id, result)

/home/ubuntu/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:11: DeprecationWarning: count is deprecated. Use Collection.count_documents instead.
  # This is added back by InteractiveShellApp.init_path()
 81%|████████  | 7911/9819 [08:13<02:04, 15.36it/s]

5f442f1325b9e6008db0a101 ['Trash']


 82%|████████▏ | 8084/9819 [08:23<01:52, 15.49it/s]

5f4504c9e806be263edf38b3 ['MarketDesk']


 83%|████████▎ | 8116/9819 [08:25<01:51, 15.25it/s]

...

 95%|█████████▌| 9341/9819 [09:42<00:32, 14.69it/s]

5f46eefae58e4a703b05c9d4 ['Trash']


 96%|█████████▋| 9452/9819 [09:49<00:24, 15.26it/s]

5f4763e69d398547c7d1efaf ['Trash']


100%|██████████| 9819/9819 [10:12<00:00, 16.03it/s]
